# Programming for Data Analytics in the Higher Diploma in Science in Data Analytics.

## Project_Superstore

**by Grainne Boyle**

This notebook contains a project that demonstrates what I have learned in this module.

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
#To ignore warnings, re: the figure layout changes, we import the warnings module.  
import warnings
warnings.filterwarnings('ignore')


In [3]:
# I had difficulty reading in the file, I was getting an error. I used chardet, a library that can detect file encodings.

#The file looked okay but there may have been unreadable characters, meaning the file was encoded with one format and your application is trying to read it using a different encoding format.
import chardet  # This imports a library that can detect file encodings

# Opens your file in binary mode ('rb') to read the raw bytes
with open('sample_superstore.csv', 'rb') as file:
    raw_data = file.read()  # Reads the entire file as raw bytes
    result = chardet.detect(raw_data)  # Analyzes the bytes to guess the encoding
    encoding = result['encoding']  # Gets the detected encoding type
    
# Uses the detected encoding to read the CSV file correctly
storedf = pd.read_csv('sample_superstore.csv', encoding=encoding)

In [4]:
print(storedf.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

In [5]:
# Change the columns to datetime
storedf['Order Date'] = pd.to_datetime(storedf['Order Date'], format='%m/%d/%Y')
storedf['Ship Date'] = pd.to_datetime(storedf['Ship Date'], format='%m/%d/%Y') 

In [6]:
# From viewing the data in excel, I decided to remove the following columns as there was either too much detail to analyse, e.g Customer ID or not enough varied detail , e.g Country is only United States.  
storedf = storedf.drop(columns=['Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Country', 'Postal Code', 'Product ID', 'Product Name'])

In [7]:
# after viewing in excel, I thought about what columns I could add to broaden my analysis:

# This added a column that calculates the time taken to process the order, how many days after the order was taken before it shipped. 
storedf.insert(loc=2, column='Order Processing Days', value=(storedf['Ship Date'] - storedf['Order Date']).dt.days)

# This enters a column that shows the month only so I can see if the ire are higher or lower ssale in certain months.

storedf.insert(loc=3, column='Month', value=storedf['Order Date'].dt.month)

# This enters a column that shows the gross margin as a percentage of sales.

storedf['Profit Margin (%)'] = (storedf['Profit'] / storedf['Sales']) * 100

# Note - I am assuming the sales figure is after the discount has been given. 
# This enters a column showing the sales price per unit(assuming after the discount)

storedf.insert(loc=13, column='SP per unit', value= (storedf['Sales'] / storedf['Quantity']))

# I am creating a function to create the final column. This column will classify the sales price per unit into a low,medium or high value category.  
def sales_cat(value):
    if value < 100:
        return "Low value"
    elif value > 1000:
        return "High value"
    else:
        return "Medium value"
    
storedf['Sales Category'] = storedf['SP per unit'].apply(sales_cat)





In [11]:
# Next, I preview the data to identify any issues that may arise and pre-process it if necessary.


# I run the list again to see what is included now and ensure that it included the columns in the correct locations:  
print(storedf.head())

# This shows the amount of rows and columns:  
storedf.shape



  Order Date  Ship Date  Order Processing Days  Month       Ship Mode  \
0 2016-11-08 2016-11-11                      3     11    Second Class   
1 2016-11-08 2016-11-11                      3     11    Second Class   
2 2016-06-12 2016-06-16                      4      6    Second Class   
3 2015-10-11 2015-10-18                      7     10  Standard Class   
4 2015-10-11 2015-10-18                      7     10  Standard Class   

     Segment             City       State Region         Category  \
0   Consumer        Henderson    Kentucky  South        Furniture   
1   Consumer        Henderson    Kentucky  South        Furniture   
2  Corporate      Los Angeles  California   West  Office Supplies   
3   Consumer  Fort Lauderdale     Florida  South        Furniture   
4   Consumer  Fort Lauderdale     Florida  South  Office Supplies   

  Sub-Category     Sales  Quantity  SP per unit  Discount    Profit  \
0    Bookcases  261.9600         2     130.9800      0.00   41.9136   
1   

In [10]:
# This shows the data types, I have converted the date columns to datetime, the other columns contain objects, integers and floats.
print(storedf.dtypes)


Order Date               datetime64[ns]
Ship Date                datetime64[ns]
Order Processing Days             int64
Month                             int32
Ship Mode                        object
Segment                          object
City                             object
State                            object
Region                           object
Category                         object
Sub-Category                     object
Sales                           float64
Quantity                          int64
SP per unit                     float64
Discount                        float64
Profit                          float64
Profit Margin (%)               float64
Sales Category                   object
dtype: object


In [ ]:
# This describes the numerical dataset:

storedf.describe(include=[np.number])

# This is useful as it highlights the main statistics. It would seem that there is no missing data, they all have 9994 rows , I will do another check on this below.   
# The order processing days average is 3.95 or 4.  
# The average selling price is $60.91 but the std deviation is $142.92 and the minimum is $0.33 showing a significant variability.
# Ths highest sales value is $22,638 and the highest selling price per unit is $3,773 which could suggest large outliers compared to the mean.
# Average discount is 16%, minimum is 0% so not all sales are discounted, and the highest discount is 80%.  

,Order Processing Days,Month,Sales,Quantity,SP per unit,Discount,Profit,Profit Margin (%)
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,3.958175,7.809686,229.858001,3.789574,60.919569,0.156203,28.656896,12.031393
std,1.747567,3.284654,623.245101,2.225110,142.927440,0.206452,234.260108,46.675435
min,0.000000,1.000000,0.444000,1.000000,0.336000,0.000000,-6599.978000,-275.000000
25%,3.000000,5.000000,17.280000,2.000000,5.470000,0.000000,1.728750,7.500000
50%,4.000000,9.000000,54.490000,3.000000,16.270000,0.200000,8.666500,27.000000
75%,5.000000,11.000000,209.940000,5.000000,63.940000,0.200000,29.364000,36.250000
max,7.000000,12.000000,22638.480000,14.000000,3773.080000,0.800000,8399.976000,50.000000


In [16]:
# This describes the non-numerical columns:  

storedf.describe(include=[object])

# At a glance, there are 49 states, 4 regions and 531 cities included.
# The top sales are in New York City and the State of California.
# There are 4 unique ship modes and the  top ship mode is the standard class and the top type of customer is a consumer.
# The top category is office supplies and the top sales category is of a low value.

,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales Category
count,9994,9994,9994,9994,9994,9994,9994,9994
unique,4,3,531,49,4,3,17,3
top,Standard Class,Consumer,New York City,California,West,Office Supplies,Binders,Low value
freq,5968,5191,915,2001,3203,6026,1523,8233


In [18]:
# This checks if there is any blank fields.

print(storedf.isnull().sum())


Order Date               0
Ship Date                0
Order Processing Days    0
Month                    0
Ship Mode                0
Segment                  0
City                     0
State                    0
Region                   0
Category                 0
Sub-Category             0
Sales                    0
Quantity                 0
SP per unit              0
Discount                 0
Profit                   0
Profit Margin (%)        0
Sales Category           0
dtype: int64


## Research

1. [Datetime](https://www.statology.org/convert-columns-to-datetime-pandas/) - Check to see how to use datetime function to convert dates so they can be used for analysis.  
2. [Chardet](https://stackoverflow.com/questions/54389780/using-chardet-to-detect-encoding)The file looked okay but there may have been unreadable characters, meaning the file was encoded with one format and your application is trying to read it using a different encoding format.  
3. [Adding Columns](https://realpython.com/pandas-dataframe/#inserting-and-deleting-columns) - adding a column to my file. I used this tutorial to add some columns relevant to my analysis.  
4. [Datetime](https://stackoverflow.com/questions/69375868/extract-month-from-datetime-column-in-pandas-dataframe) - extracting a month from the order date.  
5. [Adding Columns](https://stackoverflow.com/questions/59642338/creating-new-column-based-on-condition-on-other-column-in-pandas-dataframe) - Adding a column based on data in another column, in this case, using the Selling price per unit column, and categorising the sales by value.  
6. [Describe](https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/percentiles/a/percentiles-intro) - I used this to help understand the different columns in the describe command.